# Learning Outcomes
1. Gentle intro on CV
2. Review on some important notion of image arrays
3. Cropping (one of the data augmentation techniques employed in DL model training)
4. Slit and merge image channels
5. Point operators (basic mathematical operations) / gamma correction
    * Aims: Enhance / Reduce the contrast or illumination of the images.
6. Image blending (add 2 images together)

## setup

In [9]:
import sys
assert sys.version_info >= (3, 7)

import numpy as np
import cv2 as cv
from util_func import show_img

## review: important concepts on image arrays
Images can be broadly categorized as **grayscale** and **color** images.

| Grayscale | Color |
| -:- | -:- |
| matrix (2D array) | 3D array (channels) |
| (h, w) | (h, w, channels) |

In [2]:
img = np.zeros((2, 4), dtype = np.uint8)
print(img)

[[0 0 0 0]
 [0 0 0 0]]


In [3]:
img_bgr = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
print(img_bgr)

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [4]:
#change value in array

img[0, 1] = 30    #index of first row of array is 0
img[1, 2] = 100
print(img)

[[  0  30   0   0]
 [  0   0 100   0]]


In [5]:
img_bgr = cv.cvtColor(img, cv.COLOR_GRAY2BGR)   #all the values same will only output white, gray or black color
print(img_bgr)

[[[  0   0   0]
  [ 30  30  30]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [100 100 100]
  [  0   0   0]]]


## Accessing pixel elements

In [6]:
img = cv.imread("images/lena.jfif")

a = img[50, 70, 0]
b = img.item(50, 70, 0)
a == b

True

In [7]:
%timeit a = img[50, 70, 0]
%timeit b = img.item(50, 70, 0)

179 ns ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
172 ns ± 4 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


The takeaway the execution time is more or less the same. You are free to use either one of this.

## Numpy slicing (IMPORTANT!!!)

In [ ]:
# Extract the top lft region of the image
yc, xc = img.shape[0] // 2, img.shape[1] // 2

topleft = img[:yc, :xc]  # slicing

show_img("topleft", topleft)

In [12]:
# Extract the middle region of the image

middle = img[yc-30:yc+30, xc-30:xc+30]

show_img("middle", middle)

In [13]:
# create a white image
white = np.zeros((200, 200)) + 255
white = np.uint8(white)
show_img("white", white)

## Exercise

In [14]:
# Q2: design pattern

patch = np.zeros((30, 30), dtype=np.uint8)

patch[:10, 10:20] = 255
patch[10:20, :10] = 255
patch[10:20, 20:] = 255
patch[20:, 10:20] = 255

img = np.tile(patch, (3, 3))

show_img("img", img)

In [15]:
# Q3: extract object of interest
img = cv.imread("images/flower.jfif")

show_img("img", img)

In [16]:
[i for i in dir(cv) if i.startswith('EVENT')]

['EVENT_FLAG_ALTKEY',
 'EVENT_FLAG_CTRLKEY',
 'EVENT_FLAG_LBUTTON',
 'EVENT_FLAG_MBUTTON',
 'EVENT_FLAG_RBUTTON',
 'EVENT_FLAG_SHIFTKEY',
 'EVENT_LBUTTONDBLCLK',
 'EVENT_LBUTTONDOWN',
 'EVENT_LBUTTONUP',
 'EVENT_MBUTTONDBLCLK',
 'EVENT_MBUTTONDOWN',
 'EVENT_MBUTTONUP',
 'EVENT_MOUSEHWHEEL',
 'EVENT_MOUSEMOVE',
 'EVENT_MOUSEWHEEL',
 'EVENT_RBUTTONDBLCLK',
 'EVENT_RBUTTONDOWN',
 'EVENT_RBUTTONUP']

In [17]:
#1 way:
def select_rect(img, x, y, flags, params):
    """mouseclick callback function"""
    if event == cv.EVENT_LBUTTONDOWN:
        print((x,y))
        cv.circle(img, (x, y), 1, (0, 0, 255), -1)
        cv.imshow("img", img)

img = cv.imread("images/flower.jfif")
cv.imshow("img", img)
cv.setMouseCallback("img", select_rect)
cv.destroyAllWindows()

In [ ]:
#2 way:
bbox = cv.selectROI('flower_region', img)  #window name can set it ourselves

In [ ]:
#3 way:
# top left coordinates: (91, 38) and bottom right coordinates: (170, 120)
flower = img[38:120, 91:170]

show_img("flower", flower)

## Image cropping
In terms of operations, it is identical to slicing.

In [ ]:
img = cv.imread("images/dog.jfif")
img_copy = img.copy()

h, w = img.shape[:2]
n_vertical_grids = 3
n_horizontal_grids = 3

# row sizes and column sizes
M = int(h / n_vertical_grids)
N = int(w / n_horizontal_grids)

tiles = []

for y in range(0, h, M):
    for x in range(0, w, N):
        x1 = x + N
        y1 = y + M
        
        if x1 > w and y1 > h:
            x1 = w - 1
            y1 = h - 1
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)   #green rectangle
            tile = img[y:h, x:w]
            tiles.append(tile)
        
        elif y1 > h:
            y1 = h - 1
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            tile = img[y:h, x:x1]
            tiles.append(tile)
        
        elif x1 > w:
            x1 = w - 1
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            tile = img[y:y1, x:w]
            tiles.append(tile)
        
        else:
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            tile = img[y:y1, x:x1]
            tiles.append(tile)
            
show_img("crop", img_copy)

In [ ]:
show_img("top right", tiles[2])

## Swapping region (exercise 1) - image cropping

In [ ]:
#Q1:
img_array = np.zeros_like(img)

yc, xc = img.shape[0] // 2, img.shape[1] // 2

img_array[:yc, :xc] = img[yc:, xc:]
img_array[yc:, xc:] = img[:yc, :xc]
img_array[yc:, :xc] = img[:yc, xc:]
img_array[:yc, xc:] = img[yc:, :xc]

show_img("swap", img_array)

## Split and merge color channels

In [ ]:
b, g, r = cv.split(img)
img_merge = cv.merge((b, g, r))

In [ ]:
np.array_equal(img, img_merge)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
img = cv.imread("images/lena.jfif")

b, g, r = cv.split(img)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharey=True)
fig.suptitle("different color channels")
ax1.imshow(b, cmap=plt.cm.gray)
ax1.set(title="blue channel", xticks=[], yticks=[])
ax1.imshow(g, cmap=plt.cm.gray)
ax1.set(title="green channel", xticks=[], yticks=[])
ax1.imshow(r, cmap=plt.cm.gray)
ax1.set(title="red channel", xticks=[], yticks=[])

plt.tight_layout()
plt.show()

### Exercise no 2: color channels

In [ ]:
img = cv.imread("images/dog.jfif")

channels = cv.split(img)

colors = ("blue", "green", "red")

imgs = []

for i, mat in enumerate(channels):
    arr = np.zeros_like(img)
    arr[:, :, i] = mat
    imgs.append(arr)
    
for c, img in zip(colors, imgs):
    cv.imshow(c, img)
    
cv.waitKey(0)
cv.destroyAllWindows()

## Point operators
$$ftrans(\textbf{x}) = \alpha f(\textbf{x}) + \beta$$

In [ ]:
np.array([-2, 0, 99, 260], dtype=np.uint8)

In [ ]:
def point_op(img, alpha, beta):
    """point operators. Arguments
    1. Source image
    2. multiplier
    3. constant"""
    img = img.astype(float)
    res = alpha * img + beta
    res = np.clip(res, 0, 255)
    res = np.uint8(res)
    return res

In [ ]:
# increase the brightness and contrast of image
img = cv.imread("images/bridge.jfif")

transform = point_op(img, 2, 30)

cv.imshow("original", img)
show_img("transform", transform)

In [ ]:
transform2 = point_op(img, 0.8, -20)

show_img("img_transform", transform2)

## Gamma correction
$$O = (\frac{I}{255})^{\gamma}\times255$$

In [ ]:
gamma = 1 / 2.2

lookUpTable = np.empty((1, 256), dtype=np.uint8)
for i in range(256):
    lookUpTable[0, i] = np.clip(pow(i / 255.0, gamma) * 255, 0, 255)
    
img = cv.imread("images/mountains_prop.jpg")
res = cv.LUT(img, lookUpTable)

cv.namedWindow("original", cv.WINDOW_NORMAL)
cv.imshow("original", img)
show_img("gamma corrected", res, adjust=True)

## Image blending (add 2 images)
$$h(\textbf{x}) = \alpha f(\textbf{x}) + (1 - \alpha)g(\textbf{x}) + \beta$$

In [ ]:
img = cv.imread("images/lena.jfif")
img2 = cv.imread("images/coins.jfif")

alpha = 0.6
h, w = img.shape[:2]
img2 = cv.resize(img2, (w, h))

res = cv.addWeighted(img, alpha, img2, 1-alpha, 0)

cv.imshow("img1", img)
cv.imshow("img2", img2)
show_img("blending", res)

In [5]:
#Q1
import cv2
import numpy as np

# Define the width and height of the images
width = 400
height = 300

# Calculate the total number of pixels
total_pixels = width * height

# Generate random noise color image
color_image = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8)

# Generate random noise grayscale image
grayscale_image = np.random.randint(0, 256, (height, width), dtype=np.uint8)

# Display the color image
cv2.imshow("Color Image", color_image)
cv2.waitKey(0)

# Display the grayscale image
cv2.imshow("Grayscale Image", grayscale_image)
cv2.waitKey(0)

cv2.destroyAllWindows()

In [2]:
#Q2
import cv2
import numpy as np

def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    # Get the dimensions of the image
    height, width = img.shape[:2]

    # Calculate the size of each grid patch
    patch_width = width // num_horizontal_grid
    patch_height = height // num_vertical_grid

    # Create a copy of the image
    img_with_grid = img.copy()

    # Draw vertical grid lines
    for i in range(1, num_horizontal_grid):
        x = patch_width * i
        cv2.line(img_with_grid, (x, 0), (x, height), line_color, 1)

    # Draw horizontal grid lines
    for i in range(1, num_vertical_grid):
        y = patch_height * i
        cv2.line(img_with_grid, (0, y), (width, y), line_color, 1)

    return img_with_grid

# Load the image
image = cv2.imread("images/dog.jfif")

# Define the number of grid patches and line color
num_horizontal_grid = 3
num_vertical_grid = 2
line_color = (0, 255, 0)  # Green color (BGR format)

# Crop the image into a grid with lines
image_with_grid = crop_grid(image, num_horizontal_grid, num_vertical_grid, line_color)

# Display the image with grid
cv2.imshow("Image with Grid", image_with_grid)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
#Q3
import cv2
import numpy as np

def blend_images(image1, image2, alpha):
    # Resize the images to have the same dimensions (optional)
    image1 = cv2.resize(image1, (500, 500))
    image2 = cv2.resize(image2, (500, 500))
    
    # Perform image blending
    blended_image = cv2.addWeighted(image1, alpha, image2, 1 - alpha, 0)
    
    return blended_image

# Load the base images
image1 = cv2.imread("images/lena.jfif")
image2 = cv2.imread("images/coins.jfif")

# Create a window to display the images
cv2.namedWindow("Image Blending", cv2.WINDOW_NORMAL)

# Loop for blending images with different alpha values
for alpha in np.linspace(0, 1, 100):
    blended = blend_images(image1, image2, alpha)
    
    # Display the blended image
    cv2.imshow("Image Blending", blended)
    
    # Wait for a small delay between frames
    cv2.waitKey(30)

# Wait until a key is pressed to close the window
cv2.waitKey(0)

# Close all windows
cv2.destroyAllWindows()

In [13]:
#Q4
#Using putText to insert 'watermark' into image
img = cv.imread("images/travel_hd.jpg")

if img is None:
    raise Exception("Image not found")
    
img = cv.resize(img,None,fx=0.1,fy=0.1)

font= cv.FONT_HERSHEY_SIMPLEX
y,x = img.shape[:2]
res = cv.putText(img,"UCCC2513 watermark",(0,y-50),font,1,(255,255,255),2)
show_img("watermark image",res)